In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

In [2]:
from utils import get_cuda_info

get_cuda_info()

PyTorch version: 2.5.1+cu118
**********
_CUDA version: 
CUDA version:
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Wed_Oct_30_01:18:48_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.6, V12.6.85
Build cuda_12.6.r12.6/compiler.35059454_0

**********
CUDNN version: 90100
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 4070 Ti SUPER


## Zdobycie danych

In [3]:
from utils import load_data, convert_landmarks_to_eye_movements

all_data, all_labels = load_data('miami_deception')

In [4]:
converted_data, blinks = convert_landmarks_to_eye_movements(all_data)

## Preprocessing danych

In [5]:
from utils import preprocess_data

X_train, X_val, X_test, y_train, y_val, y_test = preprocess_data(blinks, all_labels, binarize_labels=False)

In [6]:
print(X_train.shape)
print(y_train.shape)

torch.Size([224, 1679])
torch.Size([224])


In [7]:
from utils import get_class_distribution

get_class_distribution(all_labels)

===> Class distribution <===
0: 160
1: 160
=============><=============


# MODEL TORCH

## Zbudowanie modelu ekstrakcji cech

In [8]:
import torch
import torch.nn as nn

In [21]:
class BlinkingLSTMClassifier(nn.Module):
    def __init__(self, hidden_size=64, num_layers=2, dropout=0.3):
        super(BlinkingLSTMClassifier, self).__init__()
        
        self.lstm = nn.LSTM(input_size=1,  # because each timestep is a scalar (0 or 1)
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True,
                            dropout=dropout)
        
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        # x shape: [batch, time] → need to add a feature dimension
        x = x.unsqueeze(-1)  # → [batch, time, 1]
        _, (hn, _) = self.lstm(x)  # use last hidden state
        out = self.fc(hn[-1])      # hn[-1] is last layer's hidden state
        return out  # raw logits (for BCEWithLogitsLoss)


In [22]:
from torch.optim import Adam

model = BlinkingLSTMClassifier()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=1e-4)

## Trening modelu

In [23]:
from torch.utils.tensorboard import SummaryWriter

RUNS_FOLDER_PATH = os.path.abspath('runs')
writer_path = os.path.join('runs', 'torch_lstm', 'blinking')
writer = SummaryWriter(writer_path)

In [24]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [25]:
from utils.model_functions import train_torch_model_binary

train_torch_model_binary(model, criterion, optimizer, X_train, y_train, X_val, y_val, writer=writer, batch_size=32, unbalanced=True, show_prediction_stats=False)


                                          EPOCH STATISTICS                                          
Epoch       : 1
----------------------------------------------------------------------------------------------------
                     TRAINING                                         VALIDATION                    
----------------------------------------------------------------------------------------------------
Loss        : 4.8771                                    Loss        : 1.3495
Accuracy    : 0.5134                                    Accuracy    : 0.6458
Precision   : 0.2567                                    Precision   : 0.3229
Recall      : 0.5000                                    Recall      : 0.5000
F1 Score    : 0.3392                                    F1 Score    : 0.3924
----------------------------------------------------------------------------------------------------
                                          VALIDATION EXTRA                                   

## Ewaluacja modelu

In [26]:
from utils.model_functions import eval_torch_model_binary

eval_torch_model_binary(model, criterion, X_test, y_test)


                                          EPOCH STATISTICS                                          
Epoch       : 1
----------------------------------------------------------------------------------------------------
                                             VALIDATION                                             
----------------------------------------------------------------------------------------------------
Loss        : 1.3861
Accuracy    : 0.5208
Precision   : 0.2604
Recall      : 0.5000
F1 Score    : 0.3425
----------------------------------------------------------------------------------------------------
                                          VALIDATION EXTRA                                          
TP Rate     : 1.0000                                    FP Rate     : 1.0000



# TODYNET

### Przygotowanie danych

In [15]:
X_train_np = X_train.numpy()
X_val_np = X_val.numpy()
X_test_np = X_test.numpy()
y_train_np = y_train.numpy()
y_val_np = y_val.numpy()
y_test_np = y_test.numpy()

In [16]:
TodyNet_DATA_PATH = os.path.join("..", "..", "src", "external", "TodyNet", "data", "UCR", "MIAMI_DECEPTION_EYE_MOVEMENT")

os.makedirs(TodyNet_DATA_PATH, exist_ok=True)

In [17]:
X_train_tensor = torch.tensor(X_train_np, dtype=torch.float32).unsqueeze(1)  # adding channel dimension
X_val_tensor = torch.tensor(X_val_np, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test_np, dtype=torch.float32).unsqueeze(1)

# Save the data in PyTorch (.pt) format
torch.save(X_train_tensor, os.path.join(TodyNet_DATA_PATH, 'X_train.pt'))
torch.save(X_val_tensor, os.path.join(TodyNet_DATA_PATH, 'X_valid.pt'))
torch.save(X_test_tensor, os.path.join(TodyNet_DATA_PATH, 'X.pt'))

# Save the labels in PyTorch (.pt) format
torch.save(y_train, os.path.join(TodyNet_DATA_PATH, 'y_train.pt'))
torch.save(y_val, os.path.join(TodyNet_DATA_PATH, 'y_valid.pt'))
torch.save(y_test, os.path.join(TodyNet_DATA_PATH, 'y.pt'))

In [18]:
X_train_tensor.shape

torch.Size([224, 1, 1679, 2])

### Trening modelu [pool_ratio 0.8, ponieważ rozmiar danych jest zbyt duży na 0.2]

In [ ]:
# cd .\src\external\TodyNet\src\ & python train.py --dataset='MIAMI_DECEPTION_EYE_MOVEMENT' --num_layers 1 --in_dim 16 --hidden_dim 16 --out_dim 16 --pool_ratio 0.0 --kern_size "3" --groups 1